# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3b6e687d30>
├── 'a' --> tensor([[ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206]])
└── 'x' --> <FastTreeValue 0x7f3b6e687c10>
    └── 'c' --> tensor([[ 0.7213,  1.0835, -0.1337, -0.2306],
                        [-1.4343, -1.1890,  2.6288,  1.0729],
                        [-0.6382,  0.1467,  0.5808, -1.9629]])

In [4]:
t.a

tensor([[ 0.4259,  0.0769, -1.6668],
        [-0.2683, -0.5154,  0.4206]])

In [5]:
%timeit t.a

88.3 ns ± 0.327 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3b6e687d30>
├── 'a' --> tensor([[ 1.9027,  1.8413, -1.3463],
│                   [-0.0810,  0.7662,  0.4652]])
└── 'x' --> <FastTreeValue 0x7f3b6e687c10>
    └── 'c' --> tensor([[ 0.7213,  1.0835, -0.1337, -0.2306],
                        [-1.4343, -1.1890,  2.6288,  1.0729],
                        [-0.6382,  0.1467,  0.5808, -1.9629]])

In [7]:
%timeit t.a = new_value

93.7 ns ± 0.508 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206]]),
    x: Batch(
           c: tensor([[ 0.7213,  1.0835, -0.1337, -0.2306],
                      [-1.4343, -1.1890,  2.6288,  1.0729],
                      [-0.6382,  0.1467,  0.5808, -1.9629]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4259,  0.0769, -1.6668],
        [-0.2683, -0.5154,  0.4206]])

In [11]:
%timeit b.a

80.8 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-7.2869e-01, -5.1382e-01, -1.5532e-04],
               [ 1.1427e+00, -2.8861e-01, -1.9751e+00]]),
    x: Batch(
           c: tensor([[ 0.7213,  1.0835, -0.1337, -0.2306],
                      [-1.4343, -1.1890,  2.6288,  1.0729],
                      [-0.6382,  0.1467,  0.5808, -1.9629]]),
       ),
)

In [13]:
%timeit b.a = new_value

651 ns ± 0.253 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.16 µs ± 8.73 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

180 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

178 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3a8f6da430>
├── 'a' --> tensor([[[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]],
│           
│                   [[ 0.4259,  0.0769, -1.6668],
│                    [-0.2683, -0.5154,  0.4206]]])
└── 'x' --> <FastTreeValue 0x7f3b6c61ba60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

40.3 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3b6c61bf40>
├── 'a' --> tensor([[ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206],
│                   [ 0.4259,  0.0769, -1.6668],
│                   [-0.2683, -0.5154,  0.4206]])
└── 'x' --> <FastTreeValue 0x7f3b5c1c9550>
    └── 'c' --> tensor([[ 0.7213,  1.0835, -0.1337, -0.2306],
                        [-1.4343, -1.1890,  2.6288,  1.0729],
                 

In [23]:
%timeit t_cat(trees)

37.8 µs ± 61.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.4 µs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]],
       
               [[ 0.4259,  0.0769, -1.6668],
                [-0.2683, -0.5154,  0.4206]]]),
    x: Batch(
           c: tensor([[[ 0.7213,  1.0835, -0.1337, -0.2306],
                       [-1.4343, -1.1890,  2.6288,  1.0729],
                       [-0.6382,  0.1467,  0.5808, -1.9629]],
         

In [26]:
%timeit Batch.stack(batches)

100 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206],
               [ 0.4259,  0.0769, -1.6668],
               [-0.2683, -0.5154,  0.4206]]),
    x: Batch(
           c: tensor([[ 0.7213,  1.0835, -0.1337, -0.2306],
                      [-1.4343, -1.1890,  2.6288,  1.0729],
                      [-0.6382,  0.1467,  0.5808, -1.9629],
                      [ 0.7213,  1.0835, -0.1337, -0.2306],
                      [-1.4343, -

In [28]:
%timeit Batch.cat(batches)

180 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

467 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
